# Imported useful libraries 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

!pip install snscrape
!pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# Loading Dataset

###### Label Encoding and Standard Scalar

In [ ]:
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv("/content/drive/MyDrive/Dipak/Internship/Technocolab/Twitter_stock_final_dataset.csv")
df["Date"] = pd.to_datetime(df[['Day','Month','Year']])
df.index=df.Date
# df.drop(['Day','Month','Year','Date','Total Tweets','Open','High','Low','Volume'], axis=1, inplace=True)
le=LabelEncoder()
le1=LabelEncoder()

df.StockName = le.fit_transform(df.StockName)
df.Year = le1.fit_transform(df.Year)

sc1 = StandardScaler()
sc2 = StandardScaler()
df.iloc[:,9] = sc1.fit_transform(np.array(df.iloc[:,9]).reshape(-1,1))
df.iloc[:,8] = sc2.fit_transform(np.array(df.iloc[:,8]).reshape(-1,1))

In [ ]:
df.head()

,Year,Month,Day,StockName,Positive,Negative,Neutral,Total Tweets,Close,Volume,Open,High,Low,Day_of_week,Date
Date,,,,,,,,,,,,,,,
2020-01-01,0,1,1,0,10,2,8,20,-0.866296,1.426319,74.0600,75.1500,73.7975,Wednesday,2020-01-01
2020-01-01,0,1,1,1,9,0,11,20,-0.374931,-0.602748,158.7800,160.7300,158.3300,Wednesday,2020-01-01
2020-01-01,0,1,1,4,17,3,3,23,-0.803307,-0.151065,84.9000,86.1391,84.3420,Wednesday,2020-01-01
2020-01-01,0,1,1,2,1,0,0,1,-0.953099,-0.582400,59.6875,59.9775,59.1800,Wednesday,2020-01-01
2020-01-01,0,1,1,3,1,0,1,2,-0.661423,-0.875927,109.4700,111.2100,109.2750,Wednesday,2020-01-01


In [ ]:
#Dataframe for the model evaluation
conclu = pd.DataFrame()
encoder =[]
method = []
mae1 = []
rmse1 = []
r2score1 =[]
def update():
  mae1.append(mae[-1])
  rmse1.append(rmse[-1])
  r2score1.append(r2score[-1])
  return conclu

In [ ]:
# Correlation between Close and all independent features.
import seaborn as sns
cor =df.corr().iloc[:,8]
cor

Year            0.403152
Month           0.086547
Day             0.006653
StockName       0.577297
Positive        0.540999
Negative        0.547786
Neutral         0.359389
Total Tweets    0.496966
Close           1.000000
Volume         -0.311128
Open            0.999192
High            0.999160
Low             0.999518
Name: Close, dtype: float64

In [ ]:
cor[abs(cor) > abs(0.30)]

Year            0.403152
StockName       0.577297
Positive        0.540999
Negative        0.547786
Neutral         0.359389
Total Tweets    0.496966
Close           1.000000
Volume         -0.311128
Open            0.999192
High            0.999160
Low             0.999518
Name: Close, dtype: float64

## Feature Selection for the model

In [ ]:
df = df[["Year","StockName","Positive","Negative","Neutral","Close","Volume"]]
df.head()

,Year,StockName,Positive,Negative,Neutral,Close,Volume
Date,,,,,,,
2020-01-01,0,0,10,2,8,-0.866296,1.426319
2020-01-01,0,1,9,0,11,-0.374931,-0.602748
2020-01-01,0,4,17,3,3,-0.803307,-0.151065
2020-01-01,0,2,1,0,0,-0.953099,-0.582400
2020-01-01,0,3,1,0,1,-0.661423,-0.875927


In [ ]:
# Target feature and independent features
X = np.array(df.drop(["Close"],1))
y=np.array(df.Close)

## Linear Regression

In [ ]:
#Linear regression with label encoder

from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()
print(tscv)

samples=[]
mae=[]
rmse=[]
r2score =[]

for train_index, test_index in tscv.split(X):
 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  regressor = LinearRegression()
  regressor.fit(X_train,y_train)
  y_pred = regressor.predict(X_test)
  rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
  mae.append(mean_absolute_error(y_test, y_pred))
  r2score.append(r2_score(y_test, y_pred))
  samples.append(len(train_index))
for i in range(len(samples)):
  print("Samples:",samples[i],"RMSE",rmse[i]) 


method.append('LinearRegression')
encoder.append('Label')
update()

TimeSeriesSplit(max_train_size=None, n_splits=5)
Samples: 498 RMSE 0.3606687375947129
Samples: 995 RMSE 0.559709569480944
Samples: 1492 RMSE 1.2114102431364882
Samples: 1989 RMSE 0.6107836252802381
Samples: 2486 RMSE 0.7216369739555046


""


Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
samples =[]
rmse=[]
mae = []
r2score =[]
for train_index, test_index in tscv.split(X):
 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  dtregressor = DecisionTreeRegressor()
  dtregressor.fit(X_train,y_train)
  y_pred = regressor.predict(X_test)
  rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
  mae.append(mean_absolute_error(y_test, y_pred))
  r2score.append(r2_score(y_test, y_pred))
  samples.append(len(train_index))
for i in range(len(samples)):
  print("Samples:",samples[i],"RMSE",rmse[i]) 


method.append('DecisionTreeRegressor')
encoder.append('Label')
update()

Samples: 498 RMSE 0.38329688973975695
Samples: 995 RMSE 0.36328216030979754
Samples: 1492 RMSE 0.7879048697458231
Samples: 1989 RMSE 0.5689880600606996
Samples: 2486 RMSE 0.7216369739555046


""


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
samples =[]
rmse=[]
mae =[]
r2score=[]
for train_index, test_index in tscv.split(X):
 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
  regressor.fit(X_train, y_train)
  y_pred = regressor.predict(X_test)
  rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
  mae.append(mean_absolute_error(y_test, y_pred))
  r2score.append(r2_score(y_test, y_pred))
  samples.append(len(train_index))
for i in range(len(samples)):
  print("Samples:",samples[i],"RMSE",rmse[i]) 


method.append('RandomForestRegressor')
encoder.append('Label')
update()


Samples: 498 RMSE 0.31996625195602907
Samples: 995 RMSE 0.4839446349736958
Samples: 1492 RMSE 0.9997733145464133
Samples: 1989 RMSE 0.37128518072862965
Samples: 2486 RMSE 0.294549081914819


""


Support Vector Regression

In [ ]:
# from sklearn.svm import SVR

# samples =[]
# rmse=[]
# mae =[]
# r2score =[]
# for train_index, test_index in tscv.split(X):
 
#   X_train, X_test = X[train_index], X[test_index]
#   y_train, y_test = y[train_index], y[test_index]
#   regressor = SVR(kernel = 'linear' )
#   regressor.fit(X_train, y_train)
#   y_pred = regressor.predict(X_test)
#   rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
#   mae.append(mean_absolute_error(y_test, y_pred))
#   r2score.append(r2_score(y_test, y_pred))
#   samples.append(len(train_index))
# for i in range(len(samples)):
#   print("Samples:",samples[i],"RMSE",rmse[i]) 

# method.append('SVR')
# encoder.append('Label')
# update()

KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
samples =[]
rmse=[]
mae=[]
r2score=[]
for train_index, test_index in tscv.split(X):
 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  regressor = KNeighborsRegressor()
  regressor.fit(X_train, y_train)
  y_pred = regressor.predict(X_test)
  rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
  mae.append(mean_absolute_error(y_test, y_pred))
  r2score.append(r2_score(y_test, y_pred))
  samples.append(len(train_index))
for i in range(len(samples)):
  print("Samples:",samples[i],"RMSE",rmse[i]) 

method.append('KNeighborsRegressor')
encoder.append('Label')
update()

Samples: 498 RMSE 0.3771550349541429
Samples: 995 RMSE 0.5860946773245136
Samples: 1492 RMSE 1.2707707713609961
Samples: 1989 RMSE 0.7395995086287651
Samples: 2486 RMSE 0.9243802033023104


""


In [ ]:
df.head()

,Year,StockName,Positive,Negative,Neutral,Close,Volume
Date,,,,,,,
2020-01-01,0,0,10,2,8,-0.866296,1.426319
2020-01-01,0,1,9,0,11,-0.374931,-0.602748
2020-01-01,0,4,17,3,3,-0.803307,-0.151065
2020-01-01,0,2,1,0,0,-0.953099,-0.582400
2020-01-01,0,3,1,0,1,-0.661423,-0.875927


# Using Column Encoder

In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/Dipak/Internship/Technocolab/Twitter_stock_final_dataset.csv")
# df["Date"] = pd.to_datetime(df[['Day','Month','Year']])
# df.index=df.Date
# df.drop(['Day','Month','Year','Day_of_week','Date','Total Tweets','Open','High','Low','Volume'], axis=1, inplace=True)
d = pd.DataFrame()
d = pd.get_dummies(df.StockName, prefix=None, prefix_sep='_', dummy_na=False)
df1 = pd.concat([df,d], axis=1)
df1.drop(['StockName'], axis=1, inplace=True)
# d = pd.get_dummies(df.Year, prefix=None, prefix_sep='_', dummy_na=False)
# df1 = pd.concat([df,d], axis=1)
d = pd.DataFrame()
d = pd.get_dummies(df1.Year, prefix=None, prefix_sep='_', dummy_na=False)
df1 = pd.concat([df1,d], axis=1)
df1.drop(['Year'], axis=1, inplace=True)

df1.head()

,Positive,Negative,Neutral,Close,Volume,0,1,2,3,4,0,1
Date,,,,,,,,,,,,
2020-01-01,10,2,8,-0.866296,1.426319,1,0,0,0,0,1,0
2020-01-01,9,0,11,-0.374931,-0.602748,0,1,0,0,0,1,0
2020-01-01,17,3,3,-0.803307,-0.151065,0,0,0,0,1,1,0
2020-01-01,1,0,0,-0.953099,-0.582400,0,0,1,0,0,1,0
2020-01-01,1,0,1,-0.661423,-0.875927,0,0,0,1,0,1,0


## Linear Regression

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()
print(tscv)
samples=[]
mae=[]
rmse=[]
r2score=[]
X = np.array(df1.drop(["Close"],1))
y = np.array(df1.Close)
for train_index, test_index in tscv.split(X):
 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  regressor = LinearRegression()
  regressor.fit(X_train,y_train)
  y_pred = regressor.predict(X_test)
  rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
  mae.append(mean_absolute_error(y_test, y_pred))
  r2score.append(r2_score(y_test, y_pred))
  samples.append(len(train_index))
for i in range(len(samples)):
  print("Samples:",samples[i],"RMSE",rmse[i],"  Rsquared",r2score[i]) 

method.append('LinearReression')

encoder.append('Column')
update()

TimeSeriesSplit(max_train_size=None, n_splits=5)
Samples: 498 RMSE 0.31525052297018424   Rsquared 0.1790306603118158
Samples: 995 RMSE 0.5568768482064294   Rsquared 0.006347740325476692
Samples: 1492 RMSE 1.2158012422168938   Rsquared 0.16589582499339828
Samples: 1989 RMSE 0.5220689606521594   Rsquared 0.7800755230241011
Samples: 2486 RMSE 0.5705741591158597   Rsquared 0.7341046824545463


""


Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
samples =[]
rmse=[]
mae =[]
r2score=[]
for train_index, test_index in tscv.split(X):
 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  dtregressor = DecisionTreeRegressor()
  dtregressor.fit(X_train,y_train)
  y_pred = regressor.predict(X_test)
  rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
  mae.append(mean_absolute_error(y_test, y_pred))
  r2score.append(r2_score(y_test, y_pred))
  samples.append(len(train_index))
for i in range(len(samples)):
  print("Samples:",samples[i],"RMSE",rmse[i]) 

method.append('DecisionTreeRegressor')

encoder.append('Column')
update()

Samples: 498 RMSE 0.3969403915323369
Samples: 995 RMSE 0.30385353667216547
Samples: 1492 RMSE 0.689515558229528
Samples: 1989 RMSE 0.44320713190787026
Samples: 2486 RMSE 0.5705741591158597


""


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
samples =[]
rmse=[]
mae =[]
r2score=[]
for train_index, test_index in tscv.split(X):
 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
  regressor.fit(X_train, y_train)
  y_pred = regressor.predict(X_test)
  rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
  mae.append(mean_absolute_error(y_test, y_pred))
  r2score.append(r2_score(y_test, y_pred))
  samples.append(len(train_index))
for i in range(len(samples)):
  print("Samples:",samples[i],"RMSE",rmse[i]) 

method.append('RandomForestRegressor')

encoder.append('Column')
update()

Samples: 498 RMSE 0.31620590105180985
Samples: 995 RMSE 0.48169223112059334
Samples: 1492 RMSE 0.9975243184107243
Samples: 1989 RMSE 0.37011431431967834
Samples: 2486 RMSE 0.2895465644355905


""


## Support vector regression

In [ ]:
# from sklearn.svm import SVR

# samples =[]
# rmse=[]
# mae =[]
# r2score =[]
# for train_index, test_index in tscv.split(X):
 
#   X_train, X_test = X[train_index], X[test_index]
#   y_train, y_test = y[train_index], y[test_index]
#   regressor = SVR(kernel = 'linear' )
#   regressor.fit(X_train, y_train)
#   y_pred = regressor.predict(X_test)
#   rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
#   mae.append(mean_absolute_error(y_test, y_pred))
#   r2score.append(r2_score(y_test, y_pred))
#   samples.append(len(train_index))
# for i in range(len(samples)):
#   print("Samples:",samples[i],"RMSE",rmse[i]) 


# method.append('SVR')

# encoder.append('Column')
# update()

## KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
samples =[]
rmse=[]
mae=[]
r2score=[]
for train_index, test_index in tscv.split(X):
 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  regressor = KNeighborsRegressor()
  regressor.fit(X_train, y_train)
  y_pred = regressor.predict(X_test)
  rmse.append(math.sqrt(mean_squared_error(y_test, y_pred)))
  mae.append(mean_absolute_error(y_test, y_pred))
  r2score.append(r2_score(y_test, y_pred))
  samples.append(len(train_index))
for i in range(len(samples)):
  print("Samples:",samples[i],"RMSE",rmse[i], " Rsquared",r2score[i]) 

method.append('KNeighborsRegressor')

encoder.append('Column')
update()

Samples: 498 RMSE 0.36393354069911865  Rsquared -0.09410618237301027
Samples: 995 RMSE 0.5819381645595472  Rsquared -0.08510003851496628
Samples: 1492 RMSE 1.297235571381561  Rsquared 0.050417226822954575
Samples: 1989 RMSE 0.7703957186202867  Rsquared 0.5210992220425461
Samples: 2486 RMSE 0.9628911025641482  Rsquared 0.24274728511586163


""


In [ ]:
conclu["Model"] = method
conclu["Encoder"] = encoder
conclu["MAE"] = mae1
conclu["RMSE"] = rmse1
conclu["RSquared"] = r2score1


In [ ]:
conclu

,Model,Encoder,MAE,RMSE,RSquared
0,LinearRegression,Label,0.467372,0.721637,0.574672
1,DecisionTreeRegressor,Label,0.467372,0.721637,0.574672
2,RandomForestRegressor,Label,0.240062,0.294549,0.929140
3,KNeighborsRegressor,Label,0.595342,0.924380,0.302109
4,LinearReression,Column,0.361678,0.570574,0.734105
5,DecisionTreeRegressor,Column,0.361678,0.570574,0.734105
6,RandomForestRegressor,Column,0.237836,0.289547,0.931526
7,KNeighborsRegressor,Column,0.589674,0.962891,0.242747


Random Forest Hyper Parameter tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor
samples =[]
rmse=[]
mae =[]
r2score=[]

 
X_train, X_test = X[:2486], X[2486:]
y_train, y_test = y[:2486], y[2486:]
  
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print("MAE", mean_absolute_error(y_test, y_pred))
print("RMSE",np.sqrt(mean_squared_error(y_test, y_pred)))
print("Rsquared", r2_score(y_test, y_pred))




MAE 0.23783584572195418
RMSE 0.2895465644355905
Rsquared 0.931526374357263


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.model_selection import GridSearchCV
# from pprint import pprint


# n_estimators = [5, 10, 15, 20, 25, 30, 35, 40]


# max_features = ['auto', 'sqrt']

# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)

# min_samples_split = [2, 5, 10, 15,20]

# min_samples_leaf = [1, 2, 4, 6]

# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}
# pprint(random_grid)
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestRegressor(random_state=1)
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=1, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X_train, y_train)

In [ ]:
# rf_random.best_params_
# {'bootstrap': False,
#  'max_depth': 60,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 4,
#  'min_samples_split': 15,
#  'n_estimators': 35}

In [ ]:
rf1 = RandomForestRegressor(bootstrap=False, max_depth=60, max_features='sqrt', min_samples_leaf=4, min_samples_split=15, n_estimators=35, random_state=1)
rf1.fit(X_train, y_train)
pred = rf1.predict(X_test)
print("MAE:", mean_absolute_error(pred, y_test))
print("RMSE:", mean_squared_error(pred, y_test))
print("Rsquared:",r2_score(pred, y_test))

MAE: 0.24638938869415666
RMSE: 0.09330450220716613
Rsquared: 0.9302492773668559


Grid Search CV

In [ ]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {
#     'bootstrap': [True, False],
#     'max_depth': [50,70, 90,110],
#     'max_features': [2, 3],
#     'min_samples_leaf': [ 4,6,8,10],
#     'min_samples_split': [6,8, 10, 12,14],
#     'n_estimators': [100, 200, 300, 1000,1200]
# }
# # Create a based model
# rf_1 = RandomForestRegressor(random_state=1)
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf_1, param_grid = param_grid, 
#                           cv = 2, n_jobs = -1, verbose = 2)
# grid_search.fit(X_train,y_train)

In [ ]:
# grid_search.best_params_
# {'bootstrap': True,
#  'max_depth': 50,
#  'max_features': 2,
#  'min_samples_leaf': 8,
#  'min_samples_split': 6,
#  'n_estimators': 1200}

In [ ]:
rf_2 = RandomForestRegressor(bootstrap=True, max_depth=80, max_features='sqrt', min_samples_leaf=3, min_samples_split=8, n_estimators=1000, random_state=1)
rf_2.fit(X_train, y_train)
pred = rf_2.predict(X_test)
print("MAE:", mean_absolute_error(pred, y_test))
print("RMSE:", mean_squared_error(pred, y_test))
print("Rsquared:",r2_score(pred, y_test))

MAE: 0.24034408064539275
RMSE: 0.08615294457937551
Rsquared: 0.9344356841230979


Next Day Prediction

In [ ]:
import os
main_path = "/content/drive/MyDrive/Dipak/Internship/Technocolab"
os.chdir(main_path)
os.getcwd()
from functions import *
data, date, hashtag = scraper()
df2 = data_cleaner(data, date, hashtag=hashtag)
pred_data = df2[df2.Date == date]
# Starting date: 2021-09-21
# End date : 2021-09-22

Enter staring date in YYYY-MM-DD format.2021-09-21
Enter end date in YYYY-MM-DD fromat.2021-09-22
Enter hashtag.TSLA
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 1345/1345 [00:00<00:00, 551558.36it/s]
/content/drive/MyDrive/Dipak/Internship/Technocolab/functions.py:231: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/content/drive/MyDrive/Dipak/Internship/Technocolab/functions.py:232: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/content/drive/MyDrive/Dipak/Internship/Technocolab/functions.py:233: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



[*********************100%***********************]  1 of 1 completed


In [ ]:
import numpy as np
columns=['Positive', 'Negative', 'Neutral', 'Close', 'Volume', 0, 1, 2, 3, 4, 0, 1]
pred_data1=pd.DataFrame(columns=columns)
pred_data1["Positive"] = pred_data["pos_count"]
pred_data1["Negative"] = pred_data["neg_count"]
pred_data1["Neutral"] = pred_data["neu_count"]
pred_data1["Close"] = pred_data["Close"]
pred_data1["Volume"] = pred_data["Volume"]

# pred_data1["Year"] = pred_data["Year"]
pred_data1.index = pred_data.Date

for i in range(pred_data.shape[0]):
  pred_data.Year[i]= np.int64(pred_data.Year[i])
  try:
    if pred_data["Stockname"][i] == "AAPL":
      pred_data1.iloc[:,5] = 1

    elif pred_data["Stockname"][i] == "MSFT":
      pred_data1.iloc[:,6] = 1

    elif pred_data["Stockname"][i] == "NVDA":
      pred_data1.iloc[:,7] = 1

    elif pred_data["Stockname"][i] == "TSLA":
      pred_data1.iloc[:,9] = 1

    elif pred_data["Stockname"][i] == "PYPL":
      pred_data1.iloc[:,8] = 1
  except:
    continue
  if pred_data["Year"][i] == 2020:
    pred_data1.iloc[:,10] = 1

  elif pred_data["Year"][i] == 2021:
    pred_data1.iloc[:,11] = 1




# pred_data1.StockName = le.transform(pred_data1.StockName)
# pred_data1.Year = le1.transform(pred_data1.Year)

pred_data1.iloc[:,4] = sc1.transform(np.array(pred_data1.iloc[:,4]).reshape(-1,1))
pred_data1 = pred_data1.replace (np.nan, 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# dataset for prediction
pred_data1

,Positive,Negative,Neutral,Close,Volume,0,1,2,3,4,0,1
Date,,,,,,,,,,,,
2021-09-22,34,8,44,751.94,-0.737553,0,0,0,0,1,0,1


# Prediction through best model

In [ ]:
# X = np.array(df1.drop(["Close"],1))
# y = np.array(df1.Close)
# from sklearn.ensemble import RandomForestRegressor

# test = pred_data1.drop('Close', axis=1) 
# # regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
# # regressor.fit(X, y)
# rf1.fit(X,y)
# y_pred = rf1.predict(np.array(test))

# print("Pridected Value is:",sc2.inverse_transform(y_pred))
# print("Actual Value is:",pred_data1.Close.values)
# # print("MAE", mean_absolute_error(y_test, y_pred))
# # print("RMSE",np.sqrt(mean_squared_error(y_test, y_pred)))
# # print("Rsquared", r2_score(y_test, y_pred))

### RandomForestRegressor(bootstrap=True, max_depth=80, max_features='sqrt', min_samples_leaf=3, min_samples_split=8, n_estimators=1000, random_state=1)

In [ ]:
X = np.array(df1.drop(["Close"],1))
y = np.array(df1.Close)
from sklearn.ensemble import RandomForestRegressor

test = pred_data1.drop('Close', axis=1) 

rf_2.fit(X,y)
y_pred = rf_2.predict(np.array(test))

print("Pridected Value is:",sc2.inverse_transform(y_pred))
print("Actual Value is:",pred_data1.Close.values)


Pridected Value is: [684.93267054]
Actual Value is: [751.94]
